### **INFORMATION TO BE SCRAPED**

The image below indicates the information to be scraped from the website: https://www.ycombinator.com/companies


<img width="" alt="Screenshot 2022-04-03 at 7 29 58 PM" src="https://user-images.githubusercontent.com/55639062/161443204-ae7fc423-f1d3-4512-bb56-7bef85f3691e.png">

#### STEPS
1) Launch the chrome webdriver and open the link on the webdriver.
2) Use the selenium library to scroll to the end of the page.
3) Extract the links (href) to the full information on each company and pass it into beautifulsoup.
4) Use the beautifulsoup library to scrape all of the necessary information on each company.
5) Store the scraped information in a pandas dataframe and export to a suitable format. 

#### EXTRAS
1) Resilience to network failure.
2) Multi-threading for faster runtime.

#### IMPORT LIBRARIES

In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from IPython.display import Image
from datetime import datetime as dt
import time

#### FUNCTION TO CHECK THE INTERNET CONNECTIVITY

In [2]:
import urllib.request
def internet_on():
    
    try:
        urllib.request.urlopen('https://www.google.com', timeout=2)
        return True
    except:
        return False

#### FUNCTION TO SETUP CHROME BROWSER

In [3]:
def browser_setup():
    
    # Path to the Chromedriver.
    PATH = "C:/Users/famou/.wdm/drivers/chromedriver/win32/99.0.4844.51/chromedriver.exe"

    # Change options.headless to False to show chrome browser.
    options = webdriver.ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(PATH, chrome_options=options)

    # Load url in browser.
    URL = 'https://www.ycombinator.com/companies/'
    driver.get(URL)
    return driver

#### FUNCTION TO CONFIGURE SELENIUM FOR INFINITE SCROLLING

In [4]:
def infinite_scroll(driver):
    
    try:
        # Wait for some elements to load.
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, 
                        "/html/body/div/div[2]/div/div/div[2]/div[4]/a[1]/div[1]/div/img")))
        
        
        SCROLL_PAUSE_TIME = 0.5
        
        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    except TimeoutException:
        print("Loading took too much time!!. Please refresh.")

#### FUNCTION TO GET THE URL AND LOCATION OF ALL COMPANIES

In [5]:
def get_company_url_and_country():

    homepage = browser_setup()
    infinite_scroll(homepage)
    
    page = homepage.page_source
    ycombinator = BeautifulSoup(page, 'html.parser')

    # Find all the anchor tags with class = "styles-module__company___1UVnl no-hovercard"
    # and return their "href".

    sites = []
    count = 0
    company_info = {}
    for link in ycombinator.find_all('a',
                            class_ = "styles-module__company___1UVnl no-hovercard"):
        # display the actual urls
        site = link.get('href') 
        site_mod = f"https://www.ycombinator.com{site}"
        sites.append(site_mod)
        count += 1

        hold = link.find('span', class_='styles-module__coLocation___yhKam')
        company_info['Location'] = hold.text

    # Extract the country of each company
    country = [str(c).split(",")[-1].strip() for c in Location]
    company_info['Country'] = country

    # Count number of links.
    print(f"\nCompanies total:", count)
    print('\n')
    return sites

In [14]:
homepage = browser_setup()
infinite_scroll(homepage)

page = homepage.page_source
ycombinator = BeautifulSoup(page, 'html.parser')

# Find all the anchor tags with class = "styles-module__company___1UVnl no-hovercard"
# and return their "href".

sites = []
count = 0
company_info = {}
for link in ycombinator.find_all('a',
                        class_ = "styles-module__company___1UVnl no-hovercard"):
    # display the actual urls
    site = link.get('href') 
    site_mod = f"https://www.ycombinator.com{site}"
    sites.append(site_mod)
    count += 1

    hold = link.find('span', class_='styles-module__coLocation___yhKam')
    company_info['Location'] = [hold.text]

# Extract the country of each company
country = [str(c).split(",")[-1].strip() for c in company_info['Location']]
company_info['Country'] = [country]

C:\Users\famou\AppData\Local\Temp/ipykernel_19204/3071334425.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH, chrome_options=options)
C:\Users\famou\AppData\Local\Temp/ipykernel_19204/3071334425.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(PATH, chrome_options=options)


In [15]:
company_info

{'Location': ['New York, NY, USA'], 'Country': [['USA']]}

#### FUNCTION TO SCRAPE COMPANIES DATA

In [6]:
def get_companies_info(soup):
    
    # Company name.
    select = soup.find('div', class_="font-bold text-lg")
    Company_name.append(select.text)

    # Company tag.
    hold = []
    select = soup.find_all('span',  class_='ycdc-badge')
    for j in select:
        j = j.text.replace('Y Combinator Logo', '')
        hold.append(j)

    Company_tag.append(hold)    

    # Short description.
    select = soup.find('h3', class_="sm:block md:hidden")
    Short_description.append(select.text)

    # Founded.
    select = soup.find('div', class_ = "space-y-0.5")
    select = select.next_element
    for j in select:
        if j.text != 'Founded:':
            Founded.append(j.text)

    # Team Size.
    select = select.next_sibling
    for j in select:
        if j.text != 'Team Size:':
            Team_size.append(j.text)

    # Location.
    # Check the function get_company_url_and_country.
    
    # Website.
    for j in soup.find('div', class_ = "flex flex-row items-center leading-none px-3"):
        if j.get('href') != None:
            Website.append(j.get('href'))

    # Company's social media links.
    hold = []
    for j in soup.find(class_ = "space-x-2"):
        link = j.get('href')
        hold.append(link)

    Social_media_company.append(hold)

    # Full description.
    select = soup.find('p', class_="whitespace-pre-line")
    Description.append(select.text)

#### FUNCTION TO SCRAPE FOUNDERS DATA

In [7]:
def get_founders_info(soup):

    try:
        # Active founders.
        hold = []
        founders = soup.find('div', class_='space-y-5')
        for name in founders.find_all('div', class_='leading-snug'):
            hold.append(name.div.text)
        Active_founders.append(hold)

        # Founders details.
        all_info = []
        for what in founders.find_all('div', class_='flex flex-row gap-3 items-start flex-col md:flex-row'):
            info = {}
            name = what.h3.text
            split = name.split(', ')

            info['name'] = split[0]

            if split[0] != split[-1]:
                info['role'] = split[-1]
            else:
                info['role'] = ''

            
            info['social_media'] = [link['href'] for link in what.find('div', class_='mt-1 space-x-2').find_all('a')]
            
            info['description'] = [d.text for d in what.find('p', class_='prose max-w-full whitespace-pre-line')]

            all_info.append(info)


        Founders_info.append(all_info)
        
    # Handle difference in page layouts for Founders.
    except:
        hold = []
        all_info = []

        founders = soup.find('div', class_='space-y-4')

        for founder in founders.find_all('div', class_='leading-snug'):
            
            info = {}
            name = founder.find('div', class_='font-bold').text
            info['name'] = name

            divs = [what for what in founder.find_all('div')]

            info['role'] = divs[1].text
            info['social_media'] = [link['href'] for link in founder.find('div', class_='mt-1 space-x-2').find_all('a')]

            all_info.append(info)
            hold.append(founder.find('div', class_='font-bold').text)
            
        Active_founders.append(hold)
        Founders_info.append(all_info)

#### FUNCTION TO SCRAPE ALL DATA

In [8]:
def scrape_info(url): 

    # Hold the unscraped sites.
    unscraped_sites = []

    # Pass to BeautifulSoup.
    Each_page = requests.get(url)
    soup = BeautifulSoup(Each_page.content)

    try:
        get_companies_info(soup)
        get_founders_info(soup)
    
    except Exception as e:
        print(e)
        unscraped_sites.append(url)

    return unscraped_sites

#### INITIATE PLACEHOLDER LISTS

In [9]:
Company_name = []
Company_tag = []
Short_description = []
Founded = []
Team_size = []
Location = []
Country = []
Website = [] 
Active_founders = []
Social_media_company = []
Founders_info = []
Description = []

#### **SCRAPE DATA WITHOUT MULTI-THREADING**
Scraping the data without multi-threading took approximately `13 minutes`. This is largely dependent on the internect connectivity and can vary based on the connectivity speed.

In [11]:
run = True

i = 0
# Check internet connection.
conn = internet_on()

links = get_company_url_and_country()

if conn is True:
    
    while run:

        for link in links:

            unscraped_sites = scrape_info(link)
            
            print(Company_name[i])
            print(Company_tag[i])
            print(Short_description[i])
            print(Founded[i])
            print(Team_size[i])
            print(Location[i])
            print(Website[i])
            print(Active_founders[i])
            print(Social_media_company[i])
            print(Founders_info[i])
            print(Description[i])
            print('\n')
            i += 1
            
        # Print unscraped sites.
        print(f"Sites left to scrape are:", unscraped_sites)

        # Check for unscraped sites and reinitiate scraping.
        if unscraped_sites != []:
            sites = unscraped_sites
            
        else:
            print('DONE!')
            run = False

else:
    print("Check internet connection!")

C:\Users\famou\AppData\Local\Temp/ipykernel_19204/3071334425.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH, chrome_options=options)
C:\Users\famou\AppData\Local\Temp/ipykernel_19204/3071334425.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(PATH, chrome_options=options)



Companies total: 1000


Airbnb
['W09', 'Public', 'Marketplace', 'Travel']
Book accommodations around the world.
2008
5000
San Francisco, CA, USA
http://airbnb.com
['Nathan Blecharczyk', 'Brian Chesky', 'Joe Gebbia']
['https://www.linkedin.com/company/airbnb/', 'https://twitter.com/Airbnb', 'https://www.facebook.com/airbnb/', 'https://www.crunchbase.com/organization/airbnb']
[{'name': 'Nathan Blecharczyk', 'role': 'CTO', 'social_media': ['https://twitter.com/nathanblec', 'https://www.linkedin.com/in/blecharczyk/'], 'description': ['Nathan Blecharczyk is the co-founder, Chief Strategy Officer, and Chairman of Airbnb China. Nathan plays a leading role in driving key strategic initiatives across the global business. Previously he oversaw the creation of Airbnb’s engineering, data science, and performance marketing teams. Nathan became an entrepreneur in his youth, running a business while he was in high school that sold to clients in more than 20 countries. He earned a degree in Computer 

KeyboardInterrupt: 

#### REINITIATE PLACEHOLDER LISTS

In [44]:
Company_name = []
Company_tag = []
Short_description = []
Founded = []
Team_size = []
Location = []
Website = [] 
Active_founders = []
Social_media_company = []
Founders_info = []
Description = []

#### **SCRAPE DATA WITH MULTI-THREADING**
Scraping the data with multi-threading took approximately `2 minutes,33 seconds`. A significant improvement to the former. This is also largely dependent on the internect connectivity and can vary based on the connectivity speed.

In [52]:
import concurrent.futures as cf

run = True
unscraped_sites = []
i = 0

# Check internet connection.
conn = internet_on()

# Start timer.
start_thread = dt.now()

# Get company links.
links = get_company_url()


if conn is True:
    
    while run:
        
        with cf.ThreadPoolExecutor() as executor:
            executor.map(scrape_info, links)

        # Print unscraped sites.
        print(f"Sites left to scrape are:", unscraped_sites)

        # Check for unscraped sites and reinitiate scraping.
        if unscraped_sites != []:
            links = unscraped_sites
            
        else:
            print('DONE!')
            run = False

else:
    print("Check internet connection!")

runtime_thread = (dt.now() - start_thread)
print(f'Total runtime: {runtime_thread}')

C:\Users\famou\AppData\Local\Temp/ipykernel_23848/3071334425.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH, chrome_options=options)
C:\Users\famou\AppData\Local\Temp/ipykernel_23848/3071334425.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(PATH, chrome_options=options)



Companies total: 1000


Sites left to scrape are: []
DONE!
Total runtime: 0:01:00.604493


#### CHECK LENGTH OF PLACEHOLDER LISTS

In [53]:
len(Company_name), len(Company_tag), len(Short_description), len(Founded), len(Location), len(Team_size), len(Website), len(Social_media_company), len(Founders_info), len(Active_founders), len(Description)

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

#### STORE EXTRACTED DATA IN A DATAFRAME.

In [47]:
d = zip(Company_name, Company_tag, Short_description, Founded, Location, Team_size, Website, Active_founders, Social_media_company, Founders_info, Description)
mapped = list(d)
df = pd.DataFrame(mapped, columns=['Company_name','Company_tag', 'Short_description', 'Founded', 'Location', 'Team_size', 'Website', 'Active_founders', 'Company_social_media', 'Founders_info', 'Description'])
df

,Company_name,Company_tag,Short_description,Founded,Location,Team_size,Website,Active_founders,Company_social_media,Founders_info,Description
0,Airbnb,"[W09, Public, Marketplace, Travel]",Book accommodations around the world.,2008,"San Francisco, CA, USA",5000,http://airbnb.com,"[Nathan Blecharczyk, Brian Chesky, Joe Gebbia]","[https://www.linkedin.com/company/airbnb/, htt...","[{'name': 'Nathan Blecharczyk', 'role': 'CTO',...",Founded in August of 2008 and based in San Fra...
1,Coinbase,"[S12, Public]","Buy, sell, and manage cryptocurrencies.",2012,"San Francisco, CA, USA",500,https://www.coinbase.com,[Brian Armstrong],"[https://twitter.com/coinbase, https://www.fac...","[{'name': 'Brian Armstrong', 'role': 'CEO', 's...","Founded in June of 2012, Coinbase is a digital..."
2,DoorDash,"[S13, Public, 10]",Restaurant delivery.,,"San Francisco, CA, USA",1600,http://doordash.com,"[Andy Fang, Stanley Tang, Tony Xu]","[https://www.linkedin.com/company/doordash/, h...","[{'name': 'Andy Fang', 'role': 'Founder', 'soc...","Founded in 2013, DoorDash is a San Francisco-b..."
3,GitLab,"[W15, Public, DevSecOps, Developer Tools]",A complete DevOps platform delivered as a sing...,2012,"San Francisco, CA, USA",1200,http://gitlab.com/,"[Sid Sijbrandij, Dmitriy Zaporozhets]","[https://www.linkedin.com/company/gitlab-com, ...","[{'name': 'Sid Sijbrandij', 'role': 'CEO', 'so...",GitLab is the first single application for the...
4,Dropbox,"[S07, Public]",Backup and share files in the cloud.,2008,"San Francisco, CA, USA",4000,http://dropbox.com,"[Arash Ferdowsi, Drew Houston]","[https://www.linkedin.com/company/dropbox/, ht...","[{'name': 'Arash Ferdowsi', 'role': '', 'socia...",Dropbox is building the world’s first smart wo...
...,...,...,...,...,...,...,...,...,...,...,...
995,Dime,"[S21, Active, 1]",Turn your content into unique digital collecti...,2021,"San Francisco, CA, USA",2,https://www.dime.io/,"[Aaron Barbieri-Aghib, Ben Share]","[https://www.linkedin.com/company/dimecreate/,...","[{'name': 'Aaron Barbieri-Aghib', 'role': '', ...",Dime lets creators turn their content into uni...
996,Telivy,"[S21, Active, 1]",Fastest way to purchase cyber insurance for sm...,2021,"San Francisco, CA, USA",6,https://www.telivy.com,"[Naren Sathiya, Vijay Ventrapragada]","[https://www.linkedin.com/company/telivy/, htt...","[{'name': 'Naren Sathiya', 'role': '', 'social...",Telivy’s empowers SAAS platforms and insurance...
997,Zuma,"[S21, Active, B2B, Proptech, Artifical Intelli...",We help businesses convert leads into sales in...,2021,"San Francisco, CA, USA",10,https://www.getzuma.com/,"[Kendrick Bradley, Shiv Gettu]",[https://www.crunchbase.com/organization/astro],"[{'name': 'Kendrick Bradley', 'role': 'Founder...",
998,PAYZE,"[S21, Active, B2B, Fintech]",Stripe for former Soviet Union countries,2020,"Tbilisi, Georgia",18,https://payze.io/,"[Kakha Gelashvili, Giorgi Tsurtsumia]",[https://www.linkedin.com/in/giorgitsurtsumia/...,"[{'name': 'Kakha Gelashvili', 'role': '', 'soc...","Innovator of online transactions, helping busi..."


#### EXPORT DATAFRAME TO CSV

In [48]:
df.to_csv('ycombinator.csv')